In [2]:
import csv
import pandas as pd
import spacy
import re
import string
import pyLDAvis
import pyLDAvis.sklearn
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.decomposition import LatentDirichletAllocation, NMF

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

nlp = spacy.load("en_core_web_sm")

In [3]:
df = pd.read_csv('genshintwitteronepointone.csv')
df = df.drop(columns = ['Unnamed: 0', 'outlinks', 'tcooutlinks'])

alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

def cleanurlhashtags(input):

    text = re.sub(r"http\S+", "", input)
    text = re.sub(r"#\S+", "", text)
    return text

def tostring(input):
    return str(input)

df['content'] = df['content'].apply(tostring)
df['content'] = df['content'].apply(cleanurlhashtags)
df['content'] = df['content'].map(alphanumeric).map(punc_lower)
df['content'] = df['content'].fillna(value='')

content = df['content'].to_list()

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = ENGLISH_STOP_WORDS,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(content)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(content)

lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=20, random_state=0)

In [4]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

2020-11-14 20:20:17,858 : INFO : NumExpr defaulting to 8 threads.


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6     -0.176288  0.303079       1        1  8.321896
0      0.178138 -0.025573       2        1  6.192822
4     -0.131837 -0.149905       3        1  6.140107
14     0.147603 -0.029814       4        1  5.684603
13     0.139419 -0.043206       5        1  5.417066
12     0.150818 -0.040127       6        1  5.203862
19     0.017459  0.063923       7        1  5.082592
8      0.064472 -0.021054       8        1  5.076452
1     -0.202731 -0.197220       9        1  5.012290
5      0.152062  0.019499      10        1  4.951493
7      0.086773  0.031465      11        1  4.636182
2     -0.235559 -0.087465      12        1  4.513149
17     0.118001 -0.007349      13        1  4.504827
3      0.152046  0.079491      14        1  4.475396
15    -0.169882 -0.165644      15        1  4.448765
10     0.054457  0.037498      16        1  4.221620
9      0.018267  0.027895      17        1  4.091273
16    -0.133355  0.049437      18        1  4.081545
11    -0.042116 -0.120481      19        1  4.008570
18    -0.187747  0.275551      20        1  3.935490, topic_info=              Term         Freq        Total Category  logprob  loglift
437        genshin  1808.000000  1808.000000  Default  30.0000  30.0000
551         impact  1227.000000  1227.000000  Default  29.0000  29.0000
439  genshinimpact   518.000000   518.000000  Default  28.0000  28.0000
167         childe   731.000000   731.000000  Default  27.0000  27.0000
623           live   520.000000   520.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
787           play    18.837510   320.789289  Topic20  -4.7909   0.4002
937         server    15.426162    79.344217  Topic20  -4.9907   1.5974
790        players    14.653491    54.399881  Topic20  -5.0421   1.9235
92             big    14.309024    64.338270  Topic20  -5.0659   1.7319
28             amp    14.589965   212.118932  Topic20  -5.0464   0.5583

[976 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         6  0.506570        able
0         7  0.286322        able
0        10  0.022025        able
0        11  0.176198        able
1         7  0.894724  absolutely
...     ...       ...         ...
1223      9  0.004995     zhongli
1223     13  0.024974     zhongli
1223     14  0.112383     zhongli
1223     19  0.022477     zhongli
1224     12  0.934760      zombie

[2336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 5, 15, 14, 13, 20, 9, 2, 6, 8, 3, 18, 4, 16, 11, 10, 17, 12, 19])

In [5]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
1     -0.168064  0.136112       1        1  6.409891
4     -0.180492  0.075503       2        1  6.067708
6     -0.129398 -0.167758       3        1  5.863297
0      0.161947  0.042614       4        1  5.403722
13     0.163489  0.085857       5        1  5.358882
19     0.090866  0.082991       6        1  5.171793
3      0.086805 -0.184931       7        1  5.082202
8     -0.023126  0.036799       8        1  5.007026
15    -0.199641  0.052093       9        1  4.893493
11     0.075789  0.193712      10        1  4.847961
5      0.148449 -0.017155      11        1  4.758506
7      0.033057 -0.179203      12        1  4.700716
14     0.129224  0.015249      13        1  4.693439
10    -0.026839 -0.068897      14        1  4.685653
12     0.084256  0.086217      15        1  4.664798
17     0.073953 -0.072778      16        1  4.618582
2     -0.070596 -0.076063      17        1  4.597253
16    -0.153018  0.053321      18        1  4.543859
9     -0.019668 -0.047370      19        1  4.362421
18    -0.076994 -0.046312      20        1  4.268796, topic_info=              Term        Freq       Total Category  logprob  loglift
623           live  183.000000  183.000000  Default  30.0000  30.0000
522           home   79.000000   79.000000  Default  29.0000  29.0000
439  genshinimpact  171.000000  171.000000  Default  28.0000  28.0000
679       memepact   76.000000   76.000000  Default  27.0000  27.0000
551         impact  337.000000  337.000000  Default  26.0000  26.0000
..             ...         ...         ...      ...      ...      ...
765          patch   10.691841   30.967573  Topic20  -4.5158   2.0904
551         impact   15.345216  337.564994  Topic20  -4.1544   0.0629
718            new   13.061783  149.748689  Topic20  -4.3155   0.7146
415           free   11.291594   49.791409  Topic20  -4.4612   1.6701
427           game   10.700554  148.927344  Topic20  -4.5149   0.5207

[994 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         6  0.892776  absolutely
2        14  0.881364         abt
3         8  0.150679       abyss
3         9  0.050226       abyss
3        14  0.100453       abyss
...     ...       ...         ...
1223     15  0.063235     zhongli
1223     16  0.055331     zhongli
1223     17  0.023713     zhongli
1223     19  0.007904     zhongli
1224     17  0.888447      zombie

[2582 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 7, 1, 14, 20, 4, 9, 16, 12, 6, 8, 15, 11, 13, 18, 3, 17, 10, 19])

In [6]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6      0.042126  0.462012       1        1  8.321896
0     -0.110660 -0.080007       2        1  6.192822
4      0.159186  0.071622       3        1  6.140107
14    -0.269507 -0.045187       4        1  5.684603
13    -0.296909  0.322639       5        1  5.417066
12    -0.142598  0.366493       6        1  5.203862
19     0.352084  0.124009       7        1  5.082592
8     -0.451033 -0.026422       8        1  5.076452
1      0.208408 -0.107081       9        1  5.012290
5     -0.118928  0.179082      10        1  4.951493
7     -0.350653 -0.224010      11        1  4.636182
2      0.332304 -0.212276      12        1  4.513149
17    -0.352248  0.145180      13        1  4.504827
3      0.246982 -0.426530      14        1  4.475396
15     0.444923 -0.059101      15        1  4.448765
10     0.309433  0.268334      16        1  4.221620
9      0.179892  0.361122      17        1  4.091273
16     0.077571 -0.317099      18        1  4.081545
11    -0.220757 -0.362486      19        1  4.008570
18    -0.039617 -0.440296      20        1  3.935490, topic_info=              Term         Freq        Total Category  logprob  loglift
437        genshin  1808.000000  1808.000000  Default  30.0000  30.0000
551         impact  1227.000000  1227.000000  Default  29.0000  29.0000
439  genshinimpact   518.000000   518.000000  Default  28.0000  28.0000
167         childe   731.000000   731.000000  Default  27.0000  27.0000
623           live   520.000000   520.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
787           play    18.837510   320.789289  Topic20  -4.7909   0.4002
937         server    15.426162    79.344217  Topic20  -4.9907   1.5974
790        players    14.653491    54.399881  Topic20  -5.0421   1.9235
92             big    14.309024    64.338270  Topic20  -5.0659   1.7319
28             amp    14.589965   212.118932  Topic20  -5.0464   0.5583

[976 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         6  0.506570        able
0         7  0.286322        able
0        10  0.022025        able
0        11  0.176198        able
1         7  0.894724  absolutely
...     ...       ...         ...
1223      9  0.004995     zhongli
1223     13  0.024974     zhongli
1223     14  0.112383     zhongli
1223     19  0.022477     zhongli
1224     12  0.934760      zombie

[2336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 5, 15, 14, 13, 20, 9, 2, 6, 8, 3, 18, 4, 16, 11, 10, 17, 12, 19])

In [7]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
6      -10.878068   10.538728       1        1  8.321896
0      -60.884026  -75.591820       2        1  6.192822
4       -2.336119  -53.289074       3        1  6.140107
14     -61.385319  -12.914634       4        1  5.684603
13     -28.737743  126.296997       5        1  5.417066
12       1.066606 -123.552811       6        1  5.203862
19      45.140720   -7.947869       7        1  5.082592
8     -140.795975  -98.530548       8        1  5.076452
1       61.531681  -83.240562       9        1  5.012290
5      -72.563057 -140.396881      10        1  4.951493
7       15.984491   63.578186      11        1  4.636182
2     -160.851624   68.945145      12        1  4.513149
17      85.961830   51.922935      13        1  4.504827
3      -99.020836  109.936348      14        1  4.475396
15    -176.764130   -8.681684      15        1  4.448765
10    -117.189880  -37.420200      16        1  4.221620
9      108.829918  -22.946203      17        1  4.091273
16      47.588097  121.934814      18        1  4.081545
11    -106.467834   30.176889      19        1  4.008570
18     -50.613804   59.328842      20        1  3.935490, topic_info=              Term         Freq        Total Category  logprob  loglift
437        genshin  1808.000000  1808.000000  Default  30.0000  30.0000
551         impact  1227.000000  1227.000000  Default  29.0000  29.0000
439  genshinimpact   518.000000   518.000000  Default  28.0000  28.0000
167         childe   731.000000   731.000000  Default  27.0000  27.0000
623           live   520.000000   520.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
787           play    18.837510   320.789289  Topic20  -4.7909   0.4002
937         server    15.426162    79.344217  Topic20  -4.9907   1.5974
790        players    14.653491    54.399881  Topic20  -5.0421   1.9235
92             big    14.309024    64.338270  Topic20  -5.0659   1.7319
28             amp    14.589965   212.118932  Topic20  -5.0464   0.5583

[976 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         6  0.506570        able
0         7  0.286322        able
0        10  0.022025        able
0        11  0.176198        able
1         7  0.894724  absolutely
...     ...       ...         ...
1223      9  0.004995     zhongli
1223     13  0.024974     zhongli
1223     14  0.112383     zhongli
1223     19  0.022477     zhongli
1224     12  0.934760      zombie

[2336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 1, 5, 15, 14, 13, 20, 9, 2, 6, 8, 3, 18, 4, 16, 11, 10, 17, 12, 19])